# 나만의 단축키 만들기

* 컴퓨터 환경에서 특정 키의 조합이 발생했을때 내가 원하는 동작을 하게끔 하는 프로그램
* 윈도우 전용 프로그램, 맥/리눅스 환경에선 실행 로직을 수정해야함


* 난이도 : ★★★☆☆☆☆☆☆☆
* 필요라이브러리: pynput, pypiwin32
* 다루는 내용:
    * 리스트, 딕셔너리, 집합, all()함수, 리스트 컴프리헨션

In [ ]:
from pynput.keyboard import Key, Listener, KeyCode # 키보드 모니터링을 위해 필요한 라이브러리
import win32api   # 윈도우에서 프로그램을 실행하기 위해 필요한 라이브러리

# 등록된 핫키 목록
MY_HOT_KEYS = [
    {"function1": {Key.shift, KeyCode(char='a')}},
    {"function2": {Key.ctrl_l, Key.alt_l, KeyCode(char='n')}},
    {"function3": {Key.shift_l, Key.f1}},
]

# 현재 눌려져있는 키를 저장하기 위한 집합 변수
current_keys = set()

# 단축키가 눌리면 호출될 함수
def function1():
    print("함수1")
    # 윈도우에서 프로그램을 실행하기 위해 win32api 사용
    # 리눅스/맥에선 환경에 맞는 함수로 교체해야함
    win32api.WinExec("calc.exe")

def function2():
    print("함수2")
    win32api.WinExec("notepad.exe")

def function3():
    print("함수3")
    win32api.WinExec("C:\\Program Files (x86)\\Google\\Chrome\\Application\\chrome.exe")

# 키보드가 눌렸을때 호출되는 함수
def key_pressed(key):
    print("Press {}".format(key))

    # 등록된 핫키 목록 반복
    for data in MY_HOT_KEYS:
        # 등록된 핫키는 리스트안에 딕셔너리가 있는 모양이라
        # 딕셔너리의 키와 값을 구함
        FUNCTION = list(data.keys())[0]
        KEYS = list(data.values())[0]

        # 등록된 키를 반복
        if key in KEYS:
            # 등록된 키라면 일단 현재 눌림키를 저장해놓음
            # 단축키가 한개로 동작하는게 아닌경우에는 키 조합을 판단해야하기 때문
            current_keys.add(key)

            # current_keys 에 등록된 키 전체가 단축키(KEYS) 에 모두 있다면
            # 단축키가 모두 눌린것으로 판단.
            if all(k in current_keys for k in KEYS):
                # 등록된 함수명은 문자열 상태기 때문에 이를 실행하기 위해 
                # eval 함수로 문자열을 넘겨주면 실행가능한 상태의 함수 포인터를 얻을 수 있음
                # 파이썬 함수는 일급 객체
                function = eval(FUNCTION)
                function()

# 키보드가 눌렸다 떨어졌을때 호출되는 함수
def key_released(key):
    print("Release {}".format(key))

    # 현재 띄어진 키가 눌림키 변수에 등록되어있는 상태라면 삭제
    if key in current_keys:
        current_keys.remove(key)

    # esc 키가 눌리면 키보드 감지 종료
    if key == Key.esc:
        return False

# 키보드 모니터링을 위한 리스너 등록
with Listener(on_press=key_pressed, on_release=key_released) as listener:
    listener.join()